# Sentence Transformer

In [20]:
from sentence_transformers import SentenceTransformer, util
import torch
import json
import pymongo
import spacy
from tqdm import tqdm
import faiss


In [7]:
nlp = spacy.load("en_core_web_trf")
torch.cuda.is_available()

True

In [8]:
embedder = SentenceTransformer('all-mpnet-base-v2', device='cuda')

In [9]:
data = json.load(open('dataNew.json', 'r'))

In [19]:
corpus = list()
hrefList = list()
for i in tqdm(data):
    doc = nlp(data[i]['text'])
    sents = [sent.text for sent in doc.sents]
    corpus.extend(sents)
    hrefList.extend([data[i]['link'] * len(sents)])
    # corpus.extend([data[i]['text']])
    # hrefList.extend([data[i]['link']])

100%|██████████| 18/18 [00:18<00:00,  1.04s/it]


In [17]:
len(corpus)

476

In [23]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

Batches: 100%|██████████| 15/15 [00:00<00:00, 44.85it/s]


In [24]:
top_k = min(5, len(corpus))

In [25]:
query = "how to update API by importing file"

In [26]:
query_embedding = embedder.encode(query, convert_to_tensor=True)

# FAISS

In [27]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util
import json
import time

In [33]:
CHUNK_SIZE = 10
DELIMITER = ""

In [16]:
data = json.load(open('data.json', 'r'))
corpus = list()
hrefList = list()
for i in data:
    corpus.extend(data[i]['data'])
    hrefList.extend([data[i]['href']]*len(data[i]['data']))

In [53]:
faissModel = SentenceTransformer(
    'all-mpnet-base-v2', device='cuda')
faissEmbedding = faissModel.encode(corpus, show_progress_bar=True)

Batches: 100%|██████████| 15/15 [00:00<00:00, 41.73it/s]


In [56]:
composite_index = faiss.index_factory(faissEmbedding.shape[1], "IVF100,PQ8")

composite_index.train(faissEmbedding)
composite_index.add(faissEmbedding)

WARNING clustering 476 points to 100 centroids: please provide at least 3900 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 476 points to 256 centroids: please provide at least 9984 training points


In [57]:
print('size of composite_index', composite_index.ntotal)

size of composite_index 476


In [59]:

# Get the total number of vectors in the composite index
num_vectors = composite_index.ntotal

# Set the chunk size
chunk_size = 10

# Calculate the number of chunks
num_chunks = (num_vectors + chunk_size - 1) // chunk_size

# Divide the composite index into chunks and save them separately
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, num_vectors)
    chunk_index = faiss.IndexFlatL2(composite_index.d)

    # Copy vectors from the composite index to the chunk index
    chunk_index.add(composite_index.reconstruct_n(start_idx, end_idx - start_idx))

    # Save the chunk index to disk
    filename = f"chunk_{i}.index"
    faiss.write_index(chunk_index, filename)

In [62]:
import faiss
from pymongo import MongoClient
from gridfs import GridFS
from tqdm import tqdm
import io

# MongoDB connection
client = MongoClient("mongodb://localhost:27017")
db = client["your_database_name"]
fs = GridFS(db, collection="index_chunks")

# Load the composite index from disk
# composite_index = faiss.read_index("composite_index.index")

# Get the total number of vectors in the composite index
num_vectors = composite_index.ntotal

# Set the chunk size
chunk_size = 10

# Calculate the number of chunks
num_chunks = (num_vectors + chunk_size - 1) // chunk_size

# Divide the composite index into chunks and save them in MongoDB with a progress bar
with tqdm(total=num_chunks, desc="Saving Chunks") as pbar:
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, num_vectors)

        # Create a new index for the chunk
        chunk_index = faiss.IndexFlatL2(composite_index.d)

        # Copy vectors from the composite index to the chunk index
        chunk_index.add(composite_index.reconstruct_n(start_idx, end_idx - start_idx))

        # Serialize the chunk index to bytes
        index_data = faiss.serialize_index(chunk_index)

        # Create a file-like object from the serialized index data
        index_file = io.BytesIO(index_data)

        # Save the chunk index in MongoDB using GridFS
        fs.put(index_file, filename=f"chunk_{i}.index")

        pbar.update(1)


Saving Chunks: 100%|██████████| 48/48 [00:00<00:00, 638.10it/s]


In [75]:
import faiss
from pymongo import MongoClient
from gridfs import GridFS
import io
import numpy as np

# MongoDB connection
client = MongoClient("mongodb://localhost:27017")
db = client["your_database_name"]
fs = GridFS(db, collection="index_chunks")

# Retrieve the index chunks from MongoDB and load them into Faiss indexes
index_chunks = []
for i in range(fs.find().count()):
    try:
        # Read the file data from GridFS
        index_file = fs.get_last_version(filename=f"chunk_{i}.index")

        # Load the index data from the file-like object
        index_data = index_file.read()

        index_data = np.frombuffer(index_data, dtype=np.uint8)
        
        # Load the index from the index data
        index = faiss.deserialize_index(index_data)

        # Add the index to the list of chunks
        index_chunks.append(index)
    except Exception as e:
        print(f"Error loading chunk {i}, error: {e}")
    # # Read the file data from GridFS
    # index_file = fs.get_last_version(filename=f"chunk_{i}.index")
    
    # # Load the index data from the file-like object
    # index_data = index_file.read()
    
    # index_data = np.frombuffer(index_data, dtype=np.uint8)
    
    # # Load the index from the index data
    # index = faiss.deserialize_index(index_data)
    
    # # Add the index to the list of chunks
    # index_chunks.append(index)


Error loading chunk 48, error: no version -1 for filename 'chunk_48.index'


/tmp/ipykernel_55513/2399424175.py:14: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  for i in range(fs.find().count()):


In [78]:
index_chunks[0].search(faissEmbedding[0:1], 5)

(array([[0.14830208, 0.48746973, 0.62839204, 1.1104012 , 1.2427244 ]],
       dtype=float32),
 array([[0, 2, 7, 5, 9]]))